# Elements Of Programming Interviews
## Searching Algorithms
### Track 8: 12.1, 12.4, 12.5, 12.6, 12.7, 12.8, 12.9, 12.11, 12.12

### 12.1 - Search A Sorted Array For The First Occurence Of K
>Write a method that takes a sorted array and a key and returns the index of the *first* occurence of that key in the array.

First a basic binary search

In [1]:
def binary_search(arr, k):
    s = 0; e = len(arr) - 1
    #s:start_index, e:end_index, m:middle_index
    while s <= e:
        m = int(s + (e - s) / 2)
        if arr[m] < k:
            s = m + 1
        elif arr[m] == k:
            return m
        else:
            e = m - 1
    return -1

In [2]:
binary_search(range(10), 2), binary_search(range(0,10,2), 3)

(2, -1)

I can use a basic binary search, once I find a key, then all the other equivalent keys must be directly adjacent to the curr index. The slower way to find the first index would be to loop backwards until the last occurence of the key is found. A better implementation would be to continue the binary search.

In [3]:
def first_occurence_bf(arr, k):
    s = 0; e = len(arr) - 1
    #s:start_index, e:end_index, m:middle_index
    while s <= e:
        m = int(s + (e - s) / 2)
        if arr[m] < k:
            s = m + 1
        elif arr[m] == k:
            while m > 0 and arr[m - 1] == k:
                m -= 1
            return m
        else:
            e = m - 1
    return -1

In [4]:
first_occurence_bf([0, 2, 2, 2, 2, 2, 2, 3, 4, 5, 6, 7], 2)

1

In [5]:
def first_occurence(arr, k):
    start = 0; end = len(arr) - 1
    #s:start_index, e:end_index, m:middle_index
    looking_for_first = False
    result = -1
    while start <= end:
        mid = int(start + (end - start) / 2)
        if arr[mid] < k:
            start = mid + 1
        elif arr[mid] == k:
            result = mid
            end = mid - 1
        else: #arr[mid] < k
            end = mid - 1
    return result

In [6]:
(first_occurence([1, 2, 3, 3, 3, 3, 3, 5, 6, 7, 8], 3),
 first_occurence([0], 1), first_occurence([1,1,1,1,1,1,1,1,1,1], 1),
 first_occurence([1, 2, 3, 5, 6, 6, 6, 6], 6),
 first_occurence([2]* 500000, 2)
)

(2, -1, 0, 4, 0)

The difference in speed between the two methods is very noticeable

In [7]:
%timeit first_occurence([2] * 1000000, 2)

100 loops, best of 3: 2.53 ms per loop


In [8]:
%timeit first_occurence_bf([2] * 1000000, 2)

10 loops, best of 3: 85.8 ms per loop


### 12.4 - Search A Cyclically Sorted Array
An array is said to be cyclically sorted if it is possible to cyclically shift its entries so that it becomes sorted.
For example:
>[378, 478, 550, 631, 103, 203, 220, 234, 279, 368]

is cyclically sorted--a cyclic left shift by 4 leads to a sorted array.

Design an $O(logn)$ algorithm for finding the position of the samllest element in a cyclically sorted array. Assume all elements are **distinct**. For example, for the array example above, the algorithm should return 4 (the index of 103).

First I want to find the size of the cycles in the array.

In [9]:
def find_smallest_rec(arr):
    if len(arr) == 0:
        return None
    else:
        return smallest_helper(arr, 0, len(arr) - 1)

def smallest_helper_rec(arr, left, right):
    #from IPython.core.debugger import Tracer    
    #Tracer()()
    if right - left == 1:
        return arr[right]
    else:
        mid = int(left + (right - left) / 2)
        if arr[mid] < arr[right]:
            right = mid
        elif arr[mid] > arr[right]:
            left = mid
        return smallest_helper(arr, left, right)

In [10]:
def find_smallest(arr):
    left = 0; right = len(arr) - 1
    while left < right:
        mid = int(left + (right - left) / 2)
        if arr[mid] < arr[right]:
            # min must be in [left : mid]
            right = mid
        elif arr[mid] > arr[right]:
            # min must be in [mid + 1 : right]
            left = mid + 1
    return ("Index:", left, "Val:", arr[left])

In [11]:
find_smallest([5,6,1,2]), find_smallest(list(range(50,100)) + list(range(1, 25)))

(('Index:', 2, 'Val:', 1), ('Index:', 50, 'Val:', 1))

### 12.5 - Compute The Integer Square Root
Write a program which takes a nonnegative integer and returns the largest integer whose square is less than or equal to the given integer. For exaple, 

e.g. if the input is 16, return 4;

e.g. if the input is 300, return 17, since $17^2 = 289$ and $18^2 = 324 > 300$

>obvious brute force solution is iterate through a range, calculating the square for each succesive number

In [12]:
def integer_square_root_bf(num):
    counter = 0
    while True:
        if counter ** 2 > num:
            break
        counter += 1
    return counter - 1

In [13]:
integer_square_root_bf(300), integer_square_root_bf(16)

(17, 4)

>next idea is to double the counter at every loop. Once the counter squared is greater than the passed in number, check the values between the previous counter value and the current counter value.

In [14]:
def find_root_in_range(num, start, end):
    for i in range(start, end+1):
        if i ** 2 > num:
            return i - 1
def integer_square_root_bf_2(num):
    prev = 1; counter = 1
    while True:
        if counter ** 2 > num:
            ret = find_root_in_range(num, prev, counter)
            break
        prev = counter
        counter = counter * 2
    return ret

In [15]:
#sqrt(42353243) == 6507.9369
%timeit integer_square_root_bf(42353243534555)

1 loop, best of 3: 2.29 s per loop


In [16]:
%timeit integer_square_root_bf_2(42353243534555)

1 loop, best of 3: 783 ms per loop


>next approach utilizes a few key insights
* If $x^2 > k$, then all values greater than x are not candidates
* If $x^2 < k$, then all values less than x are not candidates
* the ability to eliminate large amounts of candidates can improve preformance siginifcantly

1. Initialize a range of candidates [0, k]
2. Pick a val  in the middle $[(right - left) / 2]$
    * If $val^2 > k$, change range of candidates to [start, val-1]
    * If $val^2 < k$, change range of candidates to [val+1, end]

In [17]:
def integer_square_root(k):
    candidates = range(0, k + 1)
    left = 0; right = k
    #once the right and left indicies are adjacent, the left will point at the res
    while left <= right:
        middle = int(left + (right - left) / 2)
        #candidates[middle] equal middle
        squared = middle ** 2
        if squared > k:
            right = middle - 1
        else: # squared ** 2 <= k:
            left = middle + 1
    return left - 1

In [18]:
integer_square_root(42353243), integer_square_root(0), integer_square_root(16)

(6507, 0, 4)

The difference in preformance between these 3 approaches

In [19]:
%timeit integer_square_root_bf(42353243534555)
%timeit integer_square_root_bf_2(42353243534555)
%timeit integer_square_root(42353243534555)

1 loop, best of 3: 2.25 s per loop
1 loop, best of 3: 800 ms per loop
10000 loops, best of 3: 35.3 µs per loop


### 12.6 - Compute The Real Square Root
Square root computations can be implemented using sophisticated numerical techniques involving iterative methods and logarithms. However, if you were asked to implement a square root funciton, you would not be expceted to know these techniques.

Implement a funcition which takes as input a floating point value and returns the square root.

Approach is as follows:

In [20]:
from numpy import arange
from math import sqrt
def get_root_in_interval(k, lval, rval):
    step = (rval - lval)/ 10
    candidates = arange(lval, rval+1, step)
    left = 0; right = 9
    while left <= right:
        middle = int(left + (right - left) / 2)
        squared = candidates[middle] ** 2
        if squared > k:
            right = middle - 1
        else:
            left = middle + 1
    return candidates[left - 1], candidates[left]
            
def compute_square_root(k):
    left = 0; right = k
    #want to generate result with precision of 8
    for _ in range(8):
        left, right = get_root_in_interval(k, left, right)
    return left

In [21]:
compute_square_root(5), sqrt(5)

(2.2360679499999998, 2.23606797749979)

### 12.7 - Search In A 2D Sorted Array
Call a 2D array sorted if its rows and columns are sorted in increasing sorted order. 

See example below:


<img src='Images/sorted_matrix.png'>

Design an algorithm that takes a 2D sorted array and a number and checks wheter that number appears in the array.

In [22]:
def search_matrix_basic(matrix, k):
    for row in matrix:
        for col in row:
            if col == k:
                return True
    return False

In [23]:
matrix = [[1, 4, 7, 11, 15],
          [2, 5, 8, 12, 19],
          [3, 6, 9, 16, 22],
          [10, 13, 14, 17, 24],
          [18, 21, 23, 26, 30]]

You could do a binary search on each row in the matrix, for $O(nlogn)$ complexity

For any element $n$, at matrix[r][c]
* The elements in matrix[0:r+1][0:c+1] are smaller than $n$
* The elements in matrix[r:-1][c:-1] are bigger than $n$

In [24]:
def search_matrix_binary(matrix, k):
    for row in matrix:
        if binary_search(row, k) != -1:
            return True
    return False

In [25]:
big_matrix = [[0 for x in range(1000)] for y in range(1000)]

In [26]:
#similar to last apporach of doing binary search on each row, but we can binarily
#eliminate rows as well
def search_matrix(matrix, k):
    n_rows = len(matrix);
    n_cols = len(matrix[0])
    row = 0; col = n_cols - 1
    while row < n_rows and col >= 0:
        val = matrix[row][col]
        if val > k:
            col -= 1
        elif val < k:
            row += 1
        else:
            return True
    return False
            

In [27]:
%timeit search_matrix_basic(big_matrix, 985)
%timeit search_matrix_binary(big_matrix, 985)
%timeit search_matrix(big_matrix, 985)

10 loops, best of 3: 27.8 ms per loop
100 loops, best of 3: 5.14 ms per loop
1000 loops, best of 3: 241 µs per loop


### 12.8 - Find The Min And Max Simultaneously
Given an array of comparables, you can find either the *min* or the *max* of the elements in the array with $n-1$ comparisons, where $n$ is the length of the array. Comparing elements may be expensive. Therefore, it's natural to ask if both the *min* and the *max* can be found with less than $2(n-1)$ comparisons. 

Design an algorithm to find the *min* and *max* elements in an array.

e.g. [3, 2, 5, 1, 2, 4] returns 1 for the *min* and 5 for the *max*

In [28]:
def find_min_and_max_bf(arr):
    min_ = arr[0]
    max_ = arr[0]
    for ix in range(1,len(arr)):
        if arr[ix] < min_:
            #when updating min, don't need to check for max
            min_ = arr[ix]
        elif arr[ix] > max_:
            max_ = arr[ix]

In [29]:
def find_min_and_max(arr):
    max_ = 0; min_ = 0
    pairs = range(0, len(arr), 2) if len(arr) % 2 == 0 else range(0, len(arr) - 1, 2)
    for i in pairs:
        if not max_ and not min_:
            if arr[i] > arr[i + 1]:
                min_, max_ = arr[i + 1], arr[i]
            else:
                min_, max_ = arr[i], arr[i + 1]
        elif arr[i] < arr[i + 1]:
            max_ = max(max_, arr[i + 1])
            min_ = min(min_, arr[i])
        else:
            max_ = max(max_, arr[i])
            min_ = min(min_, arr[i + 1])
    if len(arr) % 2:
        #need to check last val
        max_ = max(max_, arr[len(arr) - 1])
        min_ = min(min_, arr[len(arr) - 1])
    return (min_, max_)

In [30]:
find_min_and_max([3,2,5,1,2,4])

(1, 5)

### 12.9 - Find Kth Largest Element
Design an algorithm for computing the $kth$ largest element in an array. Assume entries are distinct.

Brute Force- sort array and then get corresponding element

In [31]:
def kth_largest_bf(arr, k):
    """brute force solution to kth largest"""
    if k > len(arr):
        return None
    l = sorted(arr, reverse=True)
    return l[k-1]
    

Select a pivot value at random in the array.
* If there are $k-1$ elements greater than the pivot, then the pivot is $kth$ largest
* If more than $k-1$ elements greater than the pivot then discard the elements less
  than the pivot, and pick a new one greater than.
* If less than $k-1$ elements greater than pivot, pick a new one out of the values
  less than the pivot

In [32]:
kth_largest_bf([1,2,3,4,5,6,7,8,9], 5)

5

In [33]:
#come back to refactor
from random import randint
def pivot_sort(arr, left, right):
    pivot_ix = randint(left, right)
    pivot = arr[pivot_ix]
    while True:
        while arr[left] < pivot:
            left += 1
        while arr[right] > pivot:
            right -= 1
        if left < right:
            if arr[left] == pivot:
                pivot_ix = right
            elif arr[right] == pivot:
                pivot_ix = left
            arr[left], arr[right] = arr[right], arr[left]
        else:
            break
    return pivot_ix

def kth_largest(arr, k):
    left = 0
    right = len(arr) - 1
    while True:
        pivot_ix = pivot_sort(arr, left, right)
        if len(arr) - pivot_ix > k:
            #pivot is too small, repivot
            left = pivot_ix
        elif len(arr) - pivot_ix < k:
            #pivot too big, repivot
            right = pivot_ix - 1
        else:
            return arr[pivot_ix]

In [34]:
kth_largest([9, 10,100,99, 3, 4, 56, 2] , 1)

100

In [35]:
from numpy.random import randn
%timeit kth_largest_bf(randn(20000), 18)
%timeit kth_largest(randn(20000), 18)

100 loops, best of 3: 9.35 ms per loop
100 loops, best of 3: 18.6 ms per loop


### 12.11 - Find The Missing IP Address
Suppose you were given a file containing roughly one billion IP addresses, each of which is a 32-bit unisgned integer. How would you programatically find an IP address that is not in the file? Assume you have unlimited drive space, but only a few megabytes of RAM at your disposal.

Could use a hashtable, then enumerate until a missing val is found, but this requires too much overhead.

Another approach is to keep a bit array of size $2^{32}$ bits, all initialized to 0. Then loop thorugh the IP addresses, marking a 1 at each bit that correspons to the IP address in the file.

In [36]:
from bitarray import bitarray
import numpy as np
def find_missing_ip(filename):
    counter0 = 0
    counter1 = 0
    k_bit_counts = np.zeros(2**16) 
    with open(filename, 'r') as f:
        for num in f:
            k_bit_counts[int(num) >> 16] += 1
        for k_bit_prefix, count in enumerate(k_bit_counts):
            #there is a ip address w/ this prefix missing
            if count < 2 ** 16:
                missing_ip_prefix = k_bit_prefix
                break
        #use a bitarray to store the existing nums
        #w/ the missing_ip_prefix
        bitarr = bitarray(2 ** 16)
        f.seek(0)
        for num in f:
            if int(num) >> 16 == missing_ip_prefix:
                bitarr[int(num) & 65535] = True
        #find the missing val in the bit arr, and calculate the prefix prepended to val
        for i, b in enumerate(bitarr):
            if not b:
                return int((bin(missing_ip_prefix)[2:] + bin(i)[2:]), 2)
    return None

In [37]:
find_missing_ip('Misc/numbers.txt')

6

### 12.12 - Find The Duplicate And Missing Elements
You are given an array of $n$ integers, each between $0$ and $n - 1$ inclusive. Exactly one element appears twice, implying that exactly one number between $0$ and $n - 1$ is missing from the array. How would you compute the duplicate and missing numbers.

In [38]:
from bitarray import bitarray
def find_dupe_and_missing_elems_bf(arr):
    n = len(arr)
    actual_sum = sum(arr)
    expected_sum = (n * (n-1)) / 2
    bitarr = bitarray(n)
    #bitarray to mark which vals appear in the arr
    for num in arr:
        bitarr[num] = True
    missing_val = None
    #find the value that is missing
    for index, bit in enumerate(bitarr):
        if not bit:
            missing_val = index
    duplicate_value = (actual_sum + missing_val) - expected_sum
    return missing_val, duplicate_value

In [39]:
def find_dupe_and_missing_elems(arr):
    n = len(arr)
    miss_dup_sum = sum(arr)
    normal_arr_sum = (n * (n - 1) / 2)
    normal_arr = range(n)
    miss_XOR_dup = 0
    #compute the xors from both arrs
    for i in range(n):
        miss_XOR_dup ^= i ^ arr[i]
    #find the differing bits between the missing and dupe value
    #find first set bit to subset vals to check
    kth_bit = 0
    while not miss_XOR_dup & 1:
        count += 1
        miss_XOR_dup = miss_XOR_dup >> 1
    mask = 1 << kth_bit
    #now loop through both arrs, only XORing vals with the kth bit set
    #this will allow us to single out either the missing or dupe val
    miss_or_dup = 0
    for i in range(n):
        if arr[i] & mask:
            miss_or_dup ^= arr[i]
        if normal_arr[i] & mask:
            miss_or_dup ^= normal_arr[i]
    if miss_or_dup in arr:
        #miss_or_dup is duplicate
        missing_val = normal_arr_sum - (miss_dup_sum - miss_or_dup)
        duplicate_val = miss_or_dup
    else:
        #miss_or_dup is missing
        duplicate_val = (miss_dup_sum + miss_or_dup) - normal_arr_sum
        missing_val = miss_or_dup
    return missing_val, duplicate_val

In [40]:
find_dupe_and_missing_elems([1,2,3,4,5,5])

(0.0, 5)